In [1]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
import xgboost as xgb
import scipy
from sklearn.metrics import fbeta_score
from PIL import Image

In [2]:
# Set the seed 
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)

In [3]:
ROOT_PATH = 'dataset'
TRAIN_PATH = ROOT_PATH + '/' + 'train_file' + '/'
TEST_PATH = ROOT_PATH + '/' + 'test_file' + '/'

TRAIN_CSV_PATH = TRAIN_PATH + 'train_label.csv'
TEST_CSV_PATH = TEST_PATH + 'test_label.csv'
df_train = pd.read_csv(TRAIN_CSV_PATH)
df_test = pd.read_csv(TEST_CSV_PATH)

In [4]:
# Function to extract the image features
def extract_features(df, data_path):
    im_features = df.copy()

    N = len(im_features.image_name.values)

    r_mean = np.zeros(N)
    g_mean = np.zeros(N)
    b_mean = np.zeros(N)

    r_std = np.zeros(N)
    g_std = np.zeros(N)
    b_std = np.zeros(N)

    r_max = np.zeros(N)
    g_max = np.zeros(N)
    b_max = np.zeros(N)

    r_min = np.zeros(N)
    g_min = np.zeros(N)
    b_min = np.zeros(N)

    r_kurtosis = np.zeros(N)
    g_kurtosis = np.zeros(N)
    b_kurtosis = np.zeros(N)
    
    r_skewness = np.zeros(N)
    g_skewness = np.zeros(N)
    b_skewness = np.zeros(N)

    for i, image_name in enumerate(tqdm(im_features.image_name.values, miniters=1000)): 
        im = Image.open(data_path + image_name)
        im = np.array(im)[:,:,:3]

        r = im[:,:,0].ravel()
        g = im[:,:,1].ravel()
        b = im[:,:,2].ravel()
        
        r_mean[i] = np.mean(r)
        g_mean[i] = np.mean(g)
        b_mean[i] = np.mean(b)

        r_std[i] = np.std(r)
        g_std[i] = np.std(g)
        b_std[i] = np.std(b)

        r_max[i] = np.max(r)
        g_max[i] = np.max(g)
        b_max[i] = np.max(b)

        r_min[i] = np.min(r)
        g_min[i] = np.min(g)
        b_min[i] = np.min(b)

        r_kurtosis[i] = scipy.stats.kurtosis(r)
        g_kurtosis[i] = scipy.stats.kurtosis(g)
        b_kurtosis[i] = scipy.stats.kurtosis(b)
        
        r_skewness[i] = scipy.stats.skew(r)
        g_skewness[i] = scipy.stats.skew(g)
        b_skewness[i] = scipy.stats.skew(b)


    im_features['r_mean'] = r_mean
    im_features['g_mean'] = g_mean
    im_features['b_mean'] = b_mean

    im_features['rgb_mean_mean'] = (r_mean + g_mean + b_mean)/3.0

    im_features['r_std'] = r_std
    im_features['g_std'] = g_std
    im_features['b_std'] = b_std

    im_features['rgb_mean_std'] = (r_std + g_std + b_std)/3.0

    im_features['r_max'] = r_max
    im_features['g_max'] = g_max
    im_features['b_max'] = b_max

    im_features['rgb_mean_max'] = (r_max + r_max + b_max)/3.0

    im_features['r_min'] = r_min
    im_features['g_min'] = g_min
    im_features['b_min'] = b_min

    im_features['rgb_mean_min'] = (r_min + g_min + b_min)/3.0

    im_features['r_range'] = r_max - r_min
    im_features['g_range'] = g_max - g_min
    im_features['b_range'] = b_max - b_min

    im_features['r_kurtosis'] = r_kurtosis
    im_features['g_kurtosis'] = g_kurtosis
    im_features['b_kurtosis'] = b_kurtosis
    
    im_features['r_skewness'] = r_skewness
    im_features['g_skewness'] = g_skewness
    im_features['b_skewness'] = b_skewness
    
    return im_features

In [5]:
# # Extract features
# print('Extracting train features')
# train_features = extract_features(df_train, TRAIN_PATH)
# print('Extracting test features')
# test_features = extract_features(df_test, TEST_PATH)

In [6]:
# # Pickle the data
# import pickle

# train_feat = open('train_features_xgb.pickle', 'wb')
# pickle.dump(train_features, train_feat, protocol=4)
# train_feat.close()

# test_feat = open('test_features_xgb.pickle', 'wb')
# pickle.dump(test_features, test_feat, protocol=4)
# test_feat.close()

In [7]:
# Load the pickled features:
import pickle

pickle_XGtrain = open('train_features_xgb.pickle', 'rb')
train_features = pickle.load(pickle_XGtrain)

pickle_yGtrain = open('test_features_xgb.pickle', 'rb')
test_features = pickle.load(pickle_yGtrain)

In [8]:
train_features

,image_name,tags,r_mean,g_mean,b_mean,rgb_mean_mean,r_std,g_std,b_std,rgb_mean_std,...,rgb_mean_min,r_range,g_range,b_range,r_kurtosis,g_kurtosis,b_kurtosis,r_skewness,g_skewness,b_skewness
0,train_3577.jpg,haze primary,148.766541,139.191345,139.831360,142.596415,6.518982,5.068980,4.251461,5.279808,...,94.333333,78.0,60.0,53.0,1.624781,1.295214,1.887434,-0.751895,-0.690477,-0.733745
1,train_10327.jpg,clear primary,220.517136,203.796860,217.190964,213.834986,3.921799,3.366178,2.795377,3.361118,...,196.666667,38.0,28.0,30.0,0.656120,0.185518,0.505991,-0.314143,-0.056736,-0.150792
2,train_1243.jpg,clear primary water,204.231445,194.632141,210.987869,203.283819,12.544279,10.132796,6.051904,9.576326,...,129.666667,128.0,95.0,75.0,2.201598,1.264594,3.308432,-0.843753,-0.572600,-0.930052
3,train_17066.jpg,clear primary,211.725586,196.971878,199.204102,202.633855,6.247410,5.451912,3.730528,5.143283,...,170.000000,66.0,58.0,40.0,0.886477,0.548840,0.906253,-0.039352,0.155083,-0.116051
4,train_15959.jpg,clear primary,222.835907,211.518372,222.824738,219.059672,8.378670,7.839344,4.539750,6.919255,...,189.000000,67.0,57.0,41.0,0.058687,-0.042396,0.157615,0.217154,0.269355,0.103681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32378,train_7813.jpg,agriculture clear primary,209.221756,197.285767,210.975830,205.827784,11.826805,8.418947,6.459005,8.901586,...,101.666667,159.0,116.0,104.0,23.282306,14.473558,15.178514,-3.836106,-2.764308,-2.642729
32379,train_32511.jpg,clear primary,210.765640,206.845963,214.733612,210.781738,4.872695,4.387733,3.425858,4.228762,...,184.666667,52.0,45.0,33.0,0.663791,0.570556,0.455217,-0.258948,-0.195252,-0.081520
32380,train_5192.jpg,partly_cloudy primary,201.624176,186.742889,190.072342,192.813136,17.593000,16.093506,15.091996,16.259501,...,110.333333,128.0,117.0,113.0,4.469777,4.516515,5.373764,-1.873831,-1.874576,-2.188219
32381,train_12172.jpg,agriculture clear cultivation habitation prima...,193.127960,179.296692,191.246964,187.890539,10.517030,7.206180,5.668010,7.797074,...,58.666667,180.0,150.0,131.0,22.681995,32.654662,59.961953,-3.444317,-3.446622,-4.921287


In [9]:
print(train_features.shape)
print(test_features.shape)

(32383, 27)
(8096, 27)


In [10]:
test_features.isnull().any().any()

False

In [11]:
# Prepare train data
X_train = np.array(train_features.drop(['image_name', 'tags'], axis=1))
y_train = []

flatten = lambda l: [item for sublist in l for item in sublist]
labels = np.array(list(set(flatten([l.split(' ') for l in train_features['tags'].values]))))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

In [12]:
label_map

{'haze': 0,
 'cultivation': 1,
 'artisinal_mine': 2,
 'clear': 3,
 'slash_burn': 4,
 'water': 5,
 'agriculture': 6,
 'conventional_mine': 7,
 'road': 8,
 'primary': 9,
 'cloudy': 10,
 'partly_cloudy': 11,
 'bare_ground': 12,
 'blow_down': 13,
 'habitation': 14,
 'blooming': 15,
 'selective_logging': 16}

In [13]:
df_train.tags.values[0]

'haze primary'

In [14]:
# Create a bag of words for train
for tags in tqdm(df_train.tags.values, miniters=1000):
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8) # float to int

100%|██████████| 32383/32383 [00:00<00:00, 249287.69it/s]


In [15]:
n_classes = y_train.shape[1] # 17 classes in total

#Create X_test dataset
X_test = np.array(test_features.drop(['image_name', 'tags'], axis=1))


In [16]:
# Train and predict with one-vs-all strategy
train_pred = np.zeros((X_train.shape[0], n_classes)) # (num_of_train_images, n_classes=17)
y_pred = np.zeros((X_test.shape[0], n_classes)) # (num_of_test_images, n_classes=17)

In [17]:
X_train

array([[ 1.48766541e+02,  1.39191345e+02,  1.39831360e+02, ...,
        -7.51894636e-01, -6.90477450e-01, -7.33745371e-01],
       [ 2.20517136e+02,  2.03796860e+02,  2.17190964e+02, ...,
        -3.14143378e-01, -5.67362195e-02, -1.50791580e-01],
       [ 2.04231445e+02,  1.94632141e+02,  2.10987869e+02, ...,
        -8.43753262e-01, -5.72600353e-01, -9.30051760e-01],
       ...,
       [ 2.01624176e+02,  1.86742889e+02,  1.90072342e+02, ...,
        -1.87383127e+00, -1.87457569e+00, -2.18821940e+00],
       [ 1.93127960e+02,  1.79296692e+02,  1.91246964e+02, ...,
        -3.44431730e+00, -3.44662165e+00, -4.92128734e+00],
       [ 2.09308685e+02,  2.06195572e+02,  2.15342728e+02, ...,
        -3.60614349e-01, -8.87750441e-02, -1.64545360e-01]])

In [18]:
for class_i in tqdm(range(n_classes), miniters=1): 
    print("class_i =", class_i, inv_label_map[class_i])
    model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                              objective='binary:logistic', n_jobs=-1, \
                              gamma=0, min_child_weight=1, max_delta_step=0, \
                              subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                              base_score=0.5, seed=random_seed, \
                              use_label_encoder=False, eval_metric='error', \
                             )
    
    model.fit(X_train, y_train[:, class_i])
#     print("Fit done!")
    train_pred[:, class_i] = model.predict_proba(X_train)[:,1]
#     print("Pred on the train set done!")
    y_pred[:, class_i] = model.predict_proba(X_test)[:,1]
    

  0%|          | 0/17 [00:00<?, ?it/s]/Users/jihoon/anaconda3/envs/tf/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


class_i = 0 haze


  6%|▌         | 1/17 [00:03<01:03,  3.96s/it]

class_i = 1 cultivation


 12%|█▏        | 2/17 [00:07<00:59,  3.96s/it]

...

 88%|████████▊ | 15/17 [00:57<00:07,  3.76s/it]

class_i = 15 blooming


 94%|█████████▍| 16/17 [01:00<00:03,  3.57s/it]

class_i = 16 selective_logging


100%|██████████| 17/17 [01:03<00:00,  3.76s/it]


In [38]:
y_pred

array([[3.14594188e-04, 1.74364669e-03, 4.86482895e-05, ...,
        4.94009582e-04, 3.29386879e-04, 6.52341259e-05],
       [9.99807846e-04, 6.01813942e-03, 4.82067007e-05, ...,
        3.74843657e-04, 2.02221199e-04, 1.02173020e-04],
       [3.43886495e-04, 3.65004092e-01, 6.59632497e-04, ...,
        3.30020264e-02, 6.27407245e-03, 7.10797217e-03],
       ...,
       [1.70503068e-03, 1.54579303e-03, 4.82067007e-05, ...,
        9.95754497e-04, 6.86999527e-04, 2.91356147e-04],
       [3.86725031e-02, 8.37389496e-04, 1.95599336e-04, ...,
        7.69737002e-04, 9.59980898e-05, 2.21029753e-04],
       [2.76346487e-04, 1.80411933e-03, 4.77528629e-05, ...,
        6.34120021e-04, 1.81099400e-04, 8.96005295e-05]])

In [39]:
df_test

,image_name,tags,pred_tags
0,test_32752.jpg,clear primary,clear primary
1,test_7767.jpg,clear primary,clear primary
2,test_4254.jpg,agriculture clear cultivation primary,cultivation clear water agriculture primary
3,test_23119.jpg,clear primary,clear primary
4,test_22101.jpg,agriculture clear primary,clear water agriculture road primary
...,...,...,...
8091,test_14994.jpg,clear primary,clear primary
8092,test_12945.jpg,haze primary,clear primary
8093,test_9238.jpg,clear primary,clear primary
8094,test_36600.jpg,cloudy,cloudy


In [59]:
from sklearn.metrics import fbeta_score
from sklearn.preprocessing import MultiLabelBinarizer

THRES = 0.2

def calc_acc(y_pred, df_test):
    
    result = np.array(y_pred)
    result = pd.DataFrame(result, columns=labels)
    preds = []
    for i in range(result.shape[0]):
        a = result.iloc[[i]]
        a = a.apply(lambda x: x > THRES, axis=1)
        a = a.transpose()
        a = a.loc[a[i] == True]
        preds.append(' '.join(list(a.index)))
        
    df_test['pred_tags'] = preds
    test = df_test['tags'].apply(lambda x: x.split(' '))
    pred = df_test['pred_tags'].apply(lambda x: x.split(' '))
    
    mlb = MultiLabelBinarizer()
    test = mlb.fit_transform(test)
    pred = mlb.transform(pred)
    score = fbeta_score(test, pred, beta=2, average='weighted')
    return score

In [60]:
print(calc_acc(y_pred, df_test))

0.8715810746920959


In [31]:
xg_results = pd.DataFrame(y_pred, columns=labels)
xg_results.head()

,haze,cultivation,artisinal_mine,clear,slash_burn,water,agriculture,conventional_mine,road,primary,cloudy,partly_cloudy,bare_ground,blow_down,habitation,blooming,selective_logging
0,0.000315,0.001744,0.000049,0.998341,0.000114,0.008357,0.002974,0.000073,0.005614,0.999667,0.000224,0.001345,0.000403,0.000142,0.000494,0.000329,0.000065
1,0.001000,0.006018,0.000048,0.995323,0.000091,0.038274,0.009449,0.000076,0.010772,0.999574,0.000248,0.000468,0.000415,0.000097,0.000375,0.000202,0.000102
2,0.000344,0.365004,0.000660,0.890882,0.017153,0.228518,0.668510,0.000398,0.081552,0.999589,0.000235,0.067683,0.013973,0.002031,0.033002,0.006274,0.007108
3,0.000252,0.001513,0.000054,0.998472,0.000109,0.007021,0.002784,0.000063,0.005056,0.999427,0.000169,0.000546,0.000307,0.000221,0.000577,0.002860,0.000194
4,0.002394,0.085349,0.000495,0.978246,0.001995,0.512305,0.777353,0.006535,0.541024,0.992847,0.000484,0.013906,0.083064,0.000224,0.057571,0.000507,0.000713


In [32]:
y_pred

array([[3.14594188e-04, 1.74364669e-03, 4.86482895e-05, ...,
        4.94009582e-04, 3.29386879e-04, 6.52341259e-05],
       [9.99807846e-04, 6.01813942e-03, 4.82067007e-05, ...,
        3.74843657e-04, 2.02221199e-04, 1.02173020e-04],
       [3.43886495e-04, 3.65004092e-01, 6.59632497e-04, ...,
        3.30020264e-02, 6.27407245e-03, 7.10797217e-03],
       ...,
       [1.70503068e-03, 1.54579303e-03, 4.82067007e-05, ...,
        9.95754497e-04, 6.86999527e-04, 2.91356147e-04],
       [3.86725031e-02, 8.37389496e-04, 1.95599336e-04, ...,
        7.69737002e-04, 9.59980898e-05, 2.21029753e-04],
       [2.76346487e-04, 1.80411933e-03, 4.77528629e-05, ...,
        6.34120021e-04, 1.81099400e-04, 8.96005295e-05]])

In [22]:
xg_train_res = pd.DataFrame(train_pred, columns=labels)
xg_train_res.head()

,haze,cultivation,artisinal_mine,clear,slash_burn,water,agriculture,conventional_mine,road,primary,cloudy,partly_cloudy,bare_ground,blow_down,habitation,blooming,selective_logging
0,0.537098,0.159694,0.000081,0.243691,0.000354,0.134333,0.568630,0.000102,0.243954,0.980574,0.010660,0.110202,0.002239,0.000582,0.156625,0.000263,0.004613
1,0.000440,0.002222,0.000054,0.996312,0.000232,0.006140,0.003109,0.000064,0.003406,0.999122,0.001018,0.000415,0.000351,0.000179,0.000348,0.000867,0.000405
2,0.000384,0.172788,0.000069,0.983728,0.019704,0.192996,0.491435,0.000225,0.285623,0.999617,0.000185,0.015980,0.021317,0.026234,0.065269,0.022148,0.084303
3,0.000233,0.003633,0.000046,0.997905,0.000109,0.025750,0.009913,0.000076,0.005932,0.999667,0.000151,0.000568,0.000342,0.001881,0.002179,0.001709,0.000104
4,0.000351,0.005659,0.000054,0.998829,0.000296,0.028705,0.005670,0.000063,0.004608,0.999393,0.000174,0.001292,0.000979,0.000448,0.000525,0.001119,0.000194
5,0.000871,0.046233,0.000120,0.064405,0.001856,0.109918,0.043219,0.000270,0.095922,0.999540,0.000299,0.927209,0.005650,0.001139,0.006057,0.011737,0.028095
6,0.002392,0.182307,0.000061,0.734244,0.003507,0.135172,0.236053,0.000172,0.144552,0.997012,0.000181,0.207602,0.009302,0.004540,0.034107,0.008751,0.029138
7,0.000232,0.001595,0.000054,0.998593,0.000154,0.010229,0.003089,0.000063,0.003839,0.999478,0.000225,0.000417,0.000353,0.000239,0.000541,0.001356,0.000151
8,0.032685,0.001973,0.000048,0.946485,0.000116,0.024188,0.005117,0.000086,0.004193,0.996651,0.002013,0.000235,0.000570,0.000218,0.000630,0.000845,0.000773
9,0.003451,0.038144,0.003852,0.011373,0.000536,0.519069,0.169243,0.004859,0.021172,0.398304,0.360448,0.224775,0.012778,0.000094,0.010437,0.000090,0.000119


In [23]:
# xg_results.to_csv('result_xgb_0.3.csv', index=False)

In [24]:
#Pickle the result
# import pickle
res = open('result_xgb_0.5.pickle', 'wb')
pickle.dump(xg_results, res, protocol=4)
res.close()

In [25]:
#Check predicted rare labels
print("Blow downs: ",len(xg_results[xg_results['blow_down']>0.2]))
print("conventional_mine: ",len(xg_results[xg_results['conventional_mine']>0.2]))
print("selective_logging: ",len(xg_results[xg_results['selective_logging']>0.2]))
print("slash_burn: ",len(xg_results[xg_results['slash_burn']>0.2]))
print("Cultivation: ",len(xg_results[xg_results['cultivation']>0.2]))

Blow downs:  2
conventional_mine:  7
selective_logging:  18
slash_burn:  1
Cultivation:  1529


In [26]:
#create prediction flag
preds = [' '.join(labels[y_pred_row > 0.2]) for y_pred_row in y_pred]

In [27]:
#Create a  dataframe wiht test images and labels 
subm = pd.DataFrame()
subm['image_name'] = test_features.image_name.values
subm['tags'] = preds
subm[:10]

,image_name,tags
0,test_32752.jpg,clear primary
1,test_7767.jpg,clear primary
2,test_4254.jpg,cultivation clear water agriculture primary
3,test_23119.jpg,clear primary
4,test_22101.jpg,clear water agriculture road primary
5,test_26818.jpg,clear primary
6,test_25502.jpg,cultivation clear water agriculture road prima...
7,test_35301.jpg,clear water agriculture road primary partly_cl...
8,test_29796.jpg,cultivation clear agriculture road primary
9,test_36349.jpg,cultivation clear water agriculture primary


In [28]:
#Create flags for prediction flags
train_preds = [' '.join(labels[y_pred_row > 0.2]) for y_pred_row in train_pred]

In [29]:
#Create a  dataframe wiht train images and labels 
trainfrm = pd.DataFrame()
trainfrm['image_name'] = train_features.image_name.values
trainfrm['pred_tags'] = train_preds
trainfrm[:10]

,image_name,pred_tags
0,train_3577.jpg,haze clear agriculture road primary
1,train_10327.jpg,clear primary
2,train_1243.jpg,clear agriculture road primary
3,train_17066.jpg,clear primary
4,train_15959.jpg,clear primary
5,train_32931.jpg,primary partly_cloudy
6,train_4151.jpg,clear agriculture primary partly_cloudy
7,train_37844.jpg,clear primary
8,train_28283.jpg,clear primary
9,train_35805.jpg,water primary cloudy partly_cloudy


In [30]:
df_train= train
df_train[:10]

NameError: name 'train' is not defined

In [ ]:
# Join to train_df to get the original 
df_train=df_train.merge(trainfrm,on='image_name')
df_train[:5]

In [ ]:
#Create one hot encoding to compare the predicted and original labels 
df_train['tags1'] = df_train['tags'].apply(lambda x: x.split(' '))
df_train['pred_tags1'] = df_train['pred_tags'].apply(lambda x: x.split(' '))

In [ ]:
df_train_actual=df_train[['image_name','tags1']]
df_train_actual[:2]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
xgbst = MultiLabelBinarizer()

actual_results = pd.DataFrame(xgbst.fit_transform(df_train_actual['tags1']),
                   columns=xgbst.classes_,
                   index=df_train_actual.index)

actual_results

In [ ]:
df_train_pred=df_train[['image_name','pred_tags1']]
df_train_pred[:2]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
xgbst = MultiLabelBinarizer()

pred_results = pd.DataFrame(xgbst.fit_transform(df_train_pred['pred_tags1']),
                   columns=xgbst.classes_,
                   index=df_train_pred.index)

pred_results

In [ ]:
#get precision -recall report
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(actual_results, pred_results, target_names=labels))